In [87]:
#Flask 
import numpy as np
import pandas as pd
from pandas import *
import datetime as dt
from datetime import timedelta, datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from flask import Flask, jsonify
from citipy import citipy
engine = create_engine("sqlite:///desktop/dataanalysis/restaurant_ratings.db")
conn= engine.connect()
data = pd.read_sql("SELECT * FROM restaurants", conn)


In [88]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# We can view all of the classes that automap found
Base.classes.keys()

['restaurants']

In [89]:
# Save references to each table
Restaurants= Base.classes.restaurants
session = Session(engine)

In [90]:
#Queries
rest_geodatas= session.query(Restaurants.latitude,Restaurants.longitude, Restaurants.city).\
filter(Restaurants.city=="?").all()

In [91]:
#Get missing cities using coodinates
cities=[]
for rest_geodata in rest_geodatas:
    city = citipy.nearest_city(rest_geodata[0],rest_geodata[1]).city_name
    cities.append(city)
    

#Convert to df
rest= pd.DataFrame(rest_geodatas, columns=['Latitude', 'Longitude','City'])
new_column=pd.Series(cities,name='City')
rest.update(new_column)


In [92]:
df = session.query(Restaurants.name, Restaurants.city, Restaurants.food_rating).all()

In [93]:
df = pd.DataFrame(df, columns=['Restaurant Name', 'City','Allergy-Friendly Rating'])
df

,Restaurant Name,City,Allergy-Friendly Rating
0,puesto de gorditas,victoria,0
1,cafe ambar,graciano sanchez,1
2,churchs,graciano sanchez,2
3,Cafe Chaires,San Luis Potosi,2
4,McDonalds Centro,Cuernavaca,2
5,Gorditas Dona Tota,graciano sanchez,2
6,tacos de barbacoa enfrente del Tec,graciano sanchez,0
7,Hamburguesas La perica,victoria,2
8,Pollo_Frito_Buenos_Aires,victoria,1
9,carnitas_mata,victoria,2


In [94]:
myhtml=df.to_html()     
print(myhtml)
text_file = open("index.html", "w")
text_file.write(myhtml)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Restaurant Name</th>
      <th>City</th>
      <th>Allergy-Friendly Rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>puesto de gorditas</td>
      <td>victoria</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>cafe ambar</td>
      <td>graciano sanchez</td>
      <td>1</td>
    </tr>
    <tr>
      <th>2</th>
      <td>churchs</td>
      <td>graciano sanchez</td>
      <td>2</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Cafe Chaires</td>
      <td>San Luis Potosi</td>
      <td>2</td>
    </tr>
    <tr>
      <th>4</th>
      <td>McDonalds Centro</td>
      <td>Cuernavaca</td>
      <td>2</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Gorditas Dona Tota</td>
      <td>graciano sanchez</td>
      <td>2</td>
    </tr>
    <tr>
      <th>6</th>
      <td>tacos de barbacoa enfrente del Tec</td>
      <td>graciano sanchez</td>


In [109]:
city_df= pd.read_sql_query(session.query(Restaurants.name,Restaurants.city, Restaurants.food_rating).\
group_by(Restaurants.city, Restaurants.name).statement, engine)


In [110]:
cuernavaca = city_df.loc[city_df.city=='Cuernavaca'].append(city_df.loc[(city_df.city=='cuernavaca')])
jiutepec = city_df.loc[city_df.city=='Jiutepec']
slp_ = city_df.loc[(city_df.city=='San Luis Potosi')].append(city_df.loc[(city_df.city=='san luis potosi')])
soledad = city_df.loc[city_df.city=='Soledad']
gs_ = city_df.loc[city_df.city=='graciano sanchez']
victoria = city_df.loc[city_df.city=='victoria']


In [111]:
cv=cuernavaca.to_html()     
print(cv)
text_file = open("cv.html", "w")
text_file.write(cv)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>food_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Arrachela Grill</td>
      <td>Cuernavaca</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Cafeteria cenidet</td>
      <td>Cuernavaca</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Chilis Cuernavaca</td>
      <td>Cuernavaca</td>
      <td>0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>El Oceano Dorado</td>
      <td>Cuernavaca</td>
      <td>2</td>
    </tr>
    <tr>
      <th>4</th>
      <td>El cotorreo</td>
      <td>Cuernavaca</td>
      <td>1</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Kiku Cuernavaca</td>
      <td>Cuernavaca</td>
      <td>2</td>
    </tr>
    <tr>
      <th>6</th>
      <td>McDonalds Centro</td>
      <td>Cuernavaca</td>
      <td>2</td>
    </tr>
    <tr>
      <th>7</th>
     

In [112]:
jp=jiutepec.to_html()     
print(jp)
text_file = open("jp.html", "w")
text_file.write(jp)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>food_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>15</th>
      <td>Giovannis</td>
      <td>Jiutepec</td>
      <td>2</td>
    </tr>
    <tr>
      <th>16</th>
      <td>Los Vikingos</td>
      <td>Jiutepec</td>
      <td>1</td>
    </tr>
    <tr>
      <th>17</th>
      <td>Restaurant Bar Coty y Pablo</td>
      <td>Jiutepec</td>
      <td>2</td>
    </tr>
    <tr>
      <th>18</th>
      <td>Restaurant Familiar El Chino</td>
      <td>Jiutepec</td>
      <td>2</td>
    </tr>
  </tbody>
</table>


In [113]:
slp=slp_.to_html()     
print(slp)
text_file = open("slp.html", "w")
text_file.write(slp)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>food_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>19</th>
      <td>Abondance Restaurante Bar</td>
      <td>San Luis Potosi</td>
      <td>0</td>
    </tr>
    <tr>
      <th>20</th>
      <td>Cabana Huasteca</td>
      <td>San Luis Potosi</td>
      <td>2</td>
    </tr>
    <tr>
      <th>21</th>
      <td>Cafe Chaires</td>
      <td>San Luis Potosi</td>
      <td>2</td>
    </tr>
    <tr>
      <th>22</th>
      <td>Cafeteria y Restaurant El Pacifico</td>
      <td>San Luis Potosi</td>
      <td>1</td>
    </tr>
    <tr>
      <th>23</th>
      <td>Carls Jr</td>
      <td>San Luis Potosi</td>
      <td>1</td>
    </tr>
    <tr>
      <th>24</th>
      <td>Chaires</td>
      <td>San Luis Potosi</td>
      <td>2</td>
    </tr>
    <tr>
      <th>25</th>
      <td>Dominos Pizza</td>
      <td>San Luis Potosi</td>
      <td

In [114]:
sld=soledad.to_html()     
print(sld)
text_file = open("sld.html", "w")
text_file.write(sld)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>food_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>83</th>
      <td>Restaurante la Estrella de Dima</td>
      <td>Soledad</td>
      <td>2</td>
    </tr>
    <tr>
      <th>84</th>
      <td>pizza clasica</td>
      <td>Soledad</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


In [115]:
gs=gs_.to_html()     
print(gs)
text_file = open("gs.html", "w")
text_file.write(gs)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>food_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>90</th>
      <td>Gorditas Dona Tota</td>
      <td>graciano sanchez</td>
      <td>2</td>
    </tr>
    <tr>
      <th>91</th>
      <td>cafe ambar</td>
      <td>graciano sanchez</td>
      <td>1</td>
    </tr>
    <tr>
      <th>92</th>
      <td>churchs</td>
      <td>graciano sanchez</td>
      <td>2</td>
    </tr>
    <tr>
      <th>93</th>
      <td>tacos de barbacoa enfrente del Tec</td>
      <td>graciano sanchez</td>
      <td>0</td>
    </tr>
    <tr>
      <th>94</th>
      <td>tacos de la estacion</td>
      <td>graciano sanchez</td>
      <td>0</td>
    </tr>
    <tr>
      <th>95</th>
      <td>tortas hawai</td>
      <td>graciano sanchez</td>
      <td>0</td>
    </tr>
    <tr>
      <th>96</th>
      <td>vips</td>
      <td>graciano sanchez</td>
      <td>1

In [116]:
vic=victoria.to_html()     
print(vic)
text_file = open("vic.html", "w")
text_file.write(vic)
text_file.close()

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>city</th>
      <th>food_rating</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>114</th>
      <td>Carreton de Flautas y Migadas</td>
      <td>victoria</td>
      <td>0</td>
    </tr>
    <tr>
      <th>115</th>
      <td>Gorditas Dona Tota</td>
      <td>victoria</td>
      <td>2</td>
    </tr>
    <tr>
      <th>116</th>
      <td>Hamburguesas La perica</td>
      <td>victoria</td>
      <td>2</td>
    </tr>
    <tr>
      <th>117</th>
      <td>Little Cesarz</td>
      <td>victoria</td>
      <td>1</td>
    </tr>
    <tr>
      <th>118</th>
      <td>Pollo_Frito_Buenos_Aires</td>
      <td>victoria</td>
      <td>1</td>
    </tr>
    <tr>
      <th>119</th>
      <td>TACOS CORRECAMINOS</td>
      <td>victoria</td>
      <td>0</td>
    </tr>
    <tr>
      <th>120</th>
      <td>TACOS EL GUERO</td>
      <td>victoria</td>
      <td>0</td>
    </tr>
